In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

In [34]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(0)
df

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,948.6,948.6,947.6,0.0,24.7,21.1,25.6,24.6,21.1,...,74.0,80.0,82.0,3.3,1.1,-20.559167,-48.545,534.36,2020-01-01 00:00:00+00:00,0
1,0.0,949.7,949.8,948.7,0.0,23.4,20.7,24.8,23.4,21.0,...,78.0,84.0,12.0,3.2,0.8,-20.559167,-48.545,534.36,2020-01-01 01:00:00+00:00,0
2,0.0,949.8,950.2,949.7,0.0,23.7,20.0,23.9,23.4,20.8,...,79.0,80.0,338.0,3.5,1.0,-20.559167,-48.545,534.36,2020-01-01 02:00:00+00:00,0
3,0.6,949.8,949.9,949.5,0.0,23.0,20.0,23.7,23.0,20.0,...,79.0,83.0,44.0,3.5,1.1,-20.559167,-48.545,534.36,2020-01-01 03:00:00+00:00,1
4,0.2,949.2,949.8,949.2,0.0,22.3,21.6,22.9,22.0,22.4,...,83.0,96.0,358.0,3.5,1.3,-20.559167,-48.545,534.36,2020-01-01 04:00:00+00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.6,944.3,945.4,944.3,276.8,27.5,22.2,27.7,23.7,23.5,...,70.0,73.0,8.0,3.6,1.3,-20.559167,-48.545,534.36,2021-12-31 19:00:00+00:00,1
8756,0.0,944.1,944.3,944.0,139.6,27.7,22.0,28.6,26.6,22.8,...,63.0,71.0,121.0,3.1,0.5,-20.559167,-48.545,534.36,2021-12-31 20:00:00+00:00,0
8757,0.0,944.5,944.5,944.1,63.3,26.8,22.3,27.9,26.2,22.9,...,67.0,76.0,101.0,2.2,1.1,-20.559167,-48.545,534.36,2021-12-31 21:00:00+00:00,0
8758,0.0,944.8,944.9,944.5,14.0,24.6,21.5,26.8,24.6,22.3,...,74.0,83.0,129.0,4.4,1.4,-20.559167,-48.545,534.36,2021-12-31 22:00:00+00:00,0


In [6]:
df.columns

Index(['Chuva', 'Pres', 'Pres_max', 'Pres_min', 'Radiacao', 'Temp',
       'Temp_orvalho', 'Temp_max', 'Temp_min', 'Temp_orvalho_max',
       'Temp_orvalho_min', 'Umid_max', 'Umid_min', 'Umid', 'Dir_vento',
       'Rajada_vento', 'Vel_vento', 'Latitude', 'Longitude', 'Altitude',
       'datahora', 'classe_chuva'],
      dtype='object')

In [7]:
df.head(2).to_dict()

{'Chuva': {0: 0.0, 1: 0.0},
 'Pres': {0: 928.3, 1: 928.9},
 'Pres_max': {0: 928.3, 1: 928.9},
 'Pres_min': {0: 927.4, 1: 928.3},
 'Radiacao': {0: 0.0, 1: 0.0},
 'Temp': {0: 26.3, 1: 25.2},
 'Temp_orvalho': {0: 18.0, 1: 18.0},
 'Temp_max': {0: 27.4, 1: 26.4},
 'Temp_min': {0: 26.2, 1: 25.2},
 'Temp_orvalho_max': {0: 18.0, 1: 18.1},
 'Temp_orvalho_min': {0: 16.8, 1: 17.7},
 'Umid_max': {0: 60.0, 1: 64.0},
 'Umid_min': {0: 52.0, 1: 60.0},
 'Umid': {0: 60.0, 1: 64.0},
 'Dir_vento': {0: 264.0, 1: 297.0},
 'Rajada_vento': {0: 2.0, 1: 2.4},
 'Vel_vento': {0: 0.3, 1: 1.2},
 'Latitude': {0: -21.78056, 1: -21.78056},
 'Longitude': {0: -47.07528, 1: -47.07528},
 'Altitude': {0: 734.15, 1: 734.15},
 'datahora': {0: Timestamp('2020-01-01 00:00:00+0000', tz='UTC'),
  1: Timestamp('2020-01-01 01:00:00+0000', tz='UTC')},
 'classe_chuva': {0: 0, 1: 0}}

In [8]:
df.isnull().sum()

Chuva               0
Pres                0
Pres_max            0
Pres_min            0
Radiacao            0
Temp                0
Temp_orvalho        0
Temp_max            0
Temp_min            0
Temp_orvalho_max    0
Temp_orvalho_min    0
Umid_max            0
Umid_min            0
Umid                0
Dir_vento           0
Rajada_vento        0
Vel_vento           0
Latitude            0
Longitude           0
Altitude            0
datahora            0
classe_chuva        0
dtype: int64

In [9]:
def subsample_sequence(df, length):
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [10]:
df_sample = subsample_sequence(df,48)
df_sample.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
5751,0.170297,933.274235,933.510771,933.034757,363.167755,21.954069,15.015658,22.620878,21.325148,15.509442,...,64.694662,67.551839,177.570819,4.082064,1.42624,-21.78056,-47.07528,734.15,2020-08-27 15:00:00+00:00,1
5752,0.170297,933.274235,933.510771,933.034757,363.167755,21.954069,15.015658,22.620878,21.325148,15.509442,...,64.694662,67.551839,177.570819,4.082064,1.42624,-21.78056,-47.07528,734.15,2020-08-27 16:00:00+00:00,1
5753,0.170297,933.274235,933.510771,933.034757,363.167755,21.954069,15.015658,22.620878,21.325148,15.509442,...,64.694662,67.551839,177.570819,4.082064,1.42624,-21.78056,-47.07528,734.15,2020-08-27 17:00:00+00:00,1
5754,0.170297,933.274235,933.510771,933.034757,363.167755,21.954069,15.015658,22.620878,21.325148,15.509442,...,64.694662,67.551839,177.570819,4.082064,1.42624,-21.78056,-47.07528,734.15,2020-08-27 18:00:00+00:00,1
5755,0.170297,933.274235,933.510771,933.034757,363.167755,21.954069,15.015658,22.620878,21.325148,15.509442,...,64.694662,67.551839,177.570819,4.082064,1.42624,-21.78056,-47.07528,734.15,2020-08-27 19:00:00+00:00,1


In [11]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['Chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [16]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [12]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [13]:
df_train.value_counts(['Chuva'])

Chuva    
0.000000     5229
0.170297     1525
0.200000       78
0.400000       32
0.600000       19
0.800000       14
1.400000       13
1.200000       10
1.000000        9
2.000000        7
2.200000        6
2.400000        6
2.800000        6
1.800000        5
2.600000        5
3.200000        3
3.600000        3
4.200000        3
1.600000        3
5.800000        2
9.600000        2
11.600000       2
8.000000        2
14.800000       2
6.400000        2
3.000000        2
3.400000        2
4.400000        2
11.200000       2
15.800000       1
16.000000       1
15.600000       1
16.400000       1
16.800000       1
22.200000       1
14.400000       1
14.200000       1
13.000000       1
12.600000       1
12.000000       1
22.800000       1
11.400000       1
6.200000        1
10.800000       1
6.000000        1
4.000000        1
4.600000        1
4.800000        1
5.000000        1
5.200000        1
5.400000        1
6.800000        1
10.200000       1
7.400000        1
8.200000        1


In [14]:
df_test.shape

(1757, 22)

In [17]:
X_train,y_train = get_X_y(df_train,6000,72)

In [18]:
X_test,y_test = get_X_y(df_test,6000,72)

In [19]:
y_train.shape

(6000, 24)

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

# # label = LabelEncoder()
# # y_enc = label.fit_transform(y.reshape((-1,)))
# y_cat = to_categorical(y_train)
# y_cat.shape

In [20]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

2022-03-24 10:44:14.261744: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
X_train.shape

(6000, 48, 20)

In [22]:
model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(24, activation="relu"))

model.compile(loss='mse',
                  optimizer='rmsprop',
                  metrics=['mae','mse','mape'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 20)         41        
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 24)                264       
                                                                 
Total params: 3,795
Trainable params: 3,754
Non-trainable params: 41
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
150/150 [==============================] - 3s 9ms/step - loss: 1.0382 - mae: 0.1762 - mse: 1.0382 - mape: 48109232.0000 - val_loss: 0.9361 - val_mae: 0.1597 - val_mse: 0.9361 - val_mape: 42601024.0000
Epoch 2/10
150/150 [==============================] - 1s 7ms/step - loss: 1.0212 - mae: 0.1706 - mse: 1.0212 - mape: 46646192.0000 - val_loss: 0.9274 - val_mae: 0.1546 - val_mse: 0.9274 - val_mape: 38677648.0000
Epoch 3/10
150/150 [==============================] - 1s 7ms/step - loss: 1.0079 - mae: 0.1662 - mse: 1.0079 - mape: 43018968.0000 - val_loss: 0.9154 - val_mae: 0.1575 - val_mse: 0.9154 - val_mape: 43554372.0000
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 0.9881 - mae: 0.1635 - mse: 0.9881 - mape: 41497512.0000 - val_loss: 0.8946 - val_mae: 0.1573 - val_mse: 0.8946 - val_mape: 44895376.0000
Epoch 5/10
150/150 [==============================] - 1s 8ms/step - loss: 0.9655 - mae: 0.1629 - mse: 0.9655 - mape: 42236392.0000 - val_loss: 0.8770 - 

In [24]:
y_pred = model.predict(X_test)
y_pred

array([[0.        , 0.        , 0.        , ..., 0.        , 0.10024326,
        0.00075147],
       [0.        , 0.        , 0.        , ..., 0.        , 0.14188528,
        0.13275538],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.02117045,
        0.        ],
       [0.        , 0.06237773, 0.        , ..., 0.        , 0.17011654,
        0.21947347],
       [0.18895763, 0.21637022, 0.        , ..., 0.        , 0.11974698,
        0.18947965]], dtype=float32)

In [25]:
model.evaluate(X_test,y_test)

188/188 [==============================] - 1s 3ms/step - loss: 0.0073 - mae: 0.0559 - mse: 0.0073 - mape: 5707255.0000


[0.007256579119712114, 0.055865731090307236, 0.007256579119712114, 5707255.0]

In [26]:
df_y_test = pd.DataFrame(y_test).T #cada coluna sera o indice para os dias do futuro -6000 dias

In [27]:
df_y_pred = pd.DataFrame(y_pred).T

In [28]:
pd.DataFrame(y_pred).describe()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.0,6000.000000,6000.000000
mean,0.035309,0.064607,0.024661,0.158315,0.144190,0.105588,0.127033,0.071185,0.107272,0.109263,...,0.061414,0.071518,0.083331,0.045182,0.029797,0.008544,0.076892,0.0,0.075040,0.083184
std,0.049934,0.080826,0.052890,0.138744,0.110682,0.104781,0.098230,0.078974,0.109367,0.116208,...,0.071874,0.087996,0.092948,0.093659,0.075529,0.026660,0.088974,0.0,0.068640,0.088491
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.148306,0.158008,0.069747,0.150015,0.024301,0.069652,0.063617,...,0.000000,0.046769,0.082880,0.000000,0.000000,0.000000,0.000000,0.0,0.094950,0.052681
75%,0.082182,0.129355,0.003349,0.263110,0.228025,0.199677,0.198650,0.130683,0.198618,0.212103,...,0.143021,0.094353,0.104536,0.043549,0.000000,0.000000,0.173857,0.0,0.149554,0.172884
max,0.188958,0.219098,0.194380,0.503309,0.468563,0.322316,0.435470,0.280110,0.364635,0.355403,...,0.231993,0.335562,0.353716,0.361645,0.328487,0.153845,0.235962,0.0,0.172111,0.233523


In [29]:
def classe_chuva(precipitacao):
    mm=precipitacao
    if np.isnan(mm):
        chuva = "NaN"
    if mm == 0:
        chuva = 0 #'nao chove'
    elif mm >0 and mm <=5.0:
        chuva = 1 #'fraca'
    elif mm >5.0 and mm<=25.0:
        chuva = 2 #'moderada'
    else:
        chuva = 3 #'forte'
    return chuva

In [30]:
df_y_test_categorical = df_y_test.applymap(classe_chuva)
df_y_test_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
1  1  1  1  1  1  1  1  0  0  0   0   0   0   0   0   0   0   0   1   1   1   1   1     279
      0  0  0  0  0  0  0  0  0   0   0   1   1   1   1   1   1   1   1   1   1   1     275
      1  1  1  1  1  0  0  0  0   0   0   0   0   0   0   0   1   1   1   1   1   1     274
   0  0  0  0  0  0  0  0  0  0   0   1   1   1   1   1   1   1   1   1   1   1   1     273
0  0  0  0  0  1  1  1  1  1  1   1   1   1   1   1   1   1   0   0   0   0   0   0     270
               0  0  0  0  0  1   1   1   1   1   1   1   1   1   1   1   1   1   0     268
1  1  1  1  1  1  1  1  1  1  0   0   0   0   0   0   0   0   0   0   0   1   1   1     267
                  0  0  0  0  0   0   0   0   0   0   0   1   1   1   1   1   1   1     266
               0  0  0  0  0  0   0   0   0   0   0   1   1   1   1   1   1   1   1     265
0  0  0  0  0  0  0  0  0  0  0   1   1   1   1   1   1   1   1   1   1   1   1   1    

In [31]:
df_y_pred_categorical = df_y_pred.applymap(classe_chuva)
df_y_pred_categorical.T.value_counts()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   1   1   1   0   0   1   0   1   1     807
      1  1  1  1  1  1  1  1  1   1   1   1   1   1   1   0   0   0   0   0   0   0     511
1  1  1  1  1  1  1  1  1  1  1   1   1   1   1   1   1   0   0   0   0   0   0   0     492
      0  1  1  1  1  1  1  1  0   0   0   0   0   0   0   0   0   0   0   0   1   1     487
                                                                      1   0   1   1     281
                     0  0  0  0   0   0   0   0   0   0   0   0   0   1   0   1   1     274
0  1  0  1  1  0  1  0  0  0  0   0   0   0   0   0   0   0   0   0   1   0   1   1     266
   0  0  0  0  0  0  0  0  0  0   1   1   1   1   1   1   1   1   0   1   0   1   1     255
1  1  0  1  1  1  1  1  1  1  1   1   0   0   0   0   0   0   0   0   0   0   1   1     254
0  0  0  1  1  1  1  1  1  1  1   1   1   1   1   1   1   1   1   0   0   0   0   0    

In [32]:
(df_y_test_categorical == df_y_pred_categorical).sum() # o quanto acertamos por dia

0       20
1       16
2       19
3       22
4       18
        ..
5995    20
5996    22
5997    21
5998    16
5999    20
Length: 6000, dtype: int64

In [33]:
(df_y_test_categorical == df_y_pred_categorical).sum().sum()/(6000*24) #accuracy

0.7879652777777778